In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as st
%matplotlib inline
from matplotlib.pyplot import figure

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
mydata_df = pd.read_csv("seasonal_trend.csv", header=0, names=['year', 'sales'], parse_dates=[0], index_col=0)

In [ ]:
type(mydata_df)

pandas.core.frame.DataFrame

In [ ]:
mydata_df.sales

In [ ]:
mydata_df1 = mydata_df['sales']

In [ ]:
mydata_df1.head()

In [ ]:
mydatats = pd.Series(mydata_df['sales'].values, index = pd.date_range('11/01/2005', periods= 175, freq='M'))

In [ ]:
mydatats

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.figure(figsize=(12, 5))
mydatats.plot()

In [ ]:
plt.figure(figsize=(12, 5))
mydatats.plot()
plt.title('Sales of Gold from 2006 to 2020')
plt.xlabel('YEAR')
plt.ylabel('Sales of Gold')
plt.legend(['Gold'])

In [ ]:
cumsum_mydatats = np.cumsum(mydatats)
cumsum_mydatats.head(10)

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(mydatats)
plt.plot(cumsum_mydatats)
plt.title('sales of Gold')
plt.xlabel('Year')
plt.ylabel('Gold Sales figure')
plt.legend(['sales', 'cum.sales'])

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(2, 1, 1)
plt.plot(mydatats)
plt.title('Gold price')

plt.subplot(2, 1, 2)
plt.plot(cumsum_mydatats)
plt.title('Cum.Gold Sales')
plt.tight_layout()

Ewponentially Weighted Moving Average(EWMA)

In [ ]:
def plot_ewma(timeseries, alpha):
  exw_ma = timeseries.ewm(alpha=alpha).mean()

  fig = plt.figure(figsize=(12,6))
  org_line = plt.plot(timeseries, color = "blue", label = "original")
  exwma_line = plt.plot(exw_ma, color = "red", label = "EWMA")
  plt.legend(loc = "best")
  plt.title("EWMA - alpha = {}".format(alpha))
  plt.show()

In [ ]:
plot_ewma(mydatats, 0.45)

In [ ]:
def plot_ewma(timeseries, alpha):
  exw_ma = timeseries.ewm(alpha=alpha).mean()

  fig = plt.figure(figsize=(12,6))
  org_line = plt.plot(timeseries, color = "blue", label = "original")
  exwma_line = plt.plot(exw_ma, color = "red", label = "EWMA")
  plt.legend(loc = "best")
  plt.title("EWMA - alpha = " +str(alpha)+ ")")
  plt.show()

In [ ]:
plot_ewma(mydatats, 0.65)

**Seasonal Decomposition**

In [ ]:
from statsmodels.tsa.seasonal import  seasonal_decompose
decom_mydatats = seasonal_decompose(mydatats)
decom_plot = decom_mydatats.plot()

In [ ]:
decom_mydatats.trend.head(20)

In [ ]:
decom_mydatats.resid.head(20)

In [ ]:
decom_mydatats.seasonal.head(20)

In [ ]:
len(mydatats)

In [ ]:
decom_mydatats1 = seasonal_decompose(mydatats, model = 'multiplicative')
decom_plot1 = decom_mydatats1.plot()

In [ ]:
decom_mydatats1.seasonal.head(20)

In [ ]:
decom_mydatats1.trend.head(20)

In [ ]:
decom_mydatats1.resid.head(20)

STL Decomposition

In [ ]:
pip install stldecompose


In [ ]:
from stldecompose import decompose
mydatats_stl = decompose(mydatats, period = 12)
mydatats_stl.trend.head(10)

In [ ]:
mydatats_stl.seasonal.head(10)

In [ ]:
plt.figure(figsize=(12,6))
stl_plot = mydatats_stl.plot()

Find out the seasonally adjusted series

In [ ]:
mydatats_adj = mydatats - mydatats_stl.seasonal
plt.figure(figsize=(12,6))
mydatats_adj.plot()

In [ ]:
plt.figure(figsize=(12,6))
mydatats.plot()

In [ ]:
mydatats_adj1 = mydatats - mydatats_stl.trend
plt.figure(figsize=(12,4))
mydatats_adj1.plot()

In [ ]:
mydatats_adj2 = mydatats_adj - mydatats_stl.trend - mydatats_stl.seasonal
plt.figure(figsize=(12,4))
mydatats_adj2.plot()

In [ ]:
mydatats_adj2.head(15)

In [ ]:
mydatats_adj2.mean()

In [ ]:
mydatats_adj2.std()

Use this decomposed time series to forecast

In [ ]:
len(mydatats)

In [ ]:
from stldecompose import decompose, forecast
from stldecompose.forecast_funcs import (naive, drift, mean, seasonal_naive)
mydatats_stl = decompose(mydatats, period= 8)
mydatats_f = forecast(mydatats_stl, steps = 25, fc_func= mean, seasonal= True )
mydatats_f.head()

In [ ]:
len(mydatats_f1)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(mydatats, label = 'Gold Sales')
plt.plot(mydatats_f1, label = mydatats_f1.columns[0])
plt.legend()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(mydatats, label = 'Gold Sales')
plt.plot(mydatats_f1, label = 'Forecast')
plt.legend()

Holt-Winter Exponential Smoothing

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
mydatats_exp = ExponentialSmoothing(mydatats, seasonal = 'additive', seasonal_periods=12)
mydatats_expfit = mydatats_exp.fit()
mydatats_expfit.params['smoothing_level']


In [ ]:
mydatats_expfit.params['smoothing_seasonal']

Alpha = 0.035 # Alpha closer to zero implies smoother the series
Gama = 0.96 # Gama closer to 1 imlies, seasonal cpmponent is quite strong

In [ ]:
mydatats_exp1 = ExponentialSmoothing(mydatats_adj2, seasonal = 'additive', seasonal_periods=13)
mydatats_expfit1 = mydatats_exp1.fit()
mydatats_expfit1.params['smoothing_level']

In [ ]:
mydatats_expfit1.params['smoothing_seasonal']

Note: You can findout the optimum Alpha and Optimum Gama by changing Seasonal_Periods. You an also apply multiple seasonal period of data after adjusting seasonality, trend etc.... 

**Forecasting Gold Sales using Holt-Winter decomposition model**

In [ ]:
mydataexpf = mydatats_expfit.predict(start = 174, end = 200)
plt.figure(figsize=(12,4))
plt.plot(mydatats, label = "sales")
plt.plot(mydataexpf, label = "forecasted Sales")
plt.legend()

Compare Forecast VS Actual data

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(mydatats, label = 'Original Sales')
plt.plot(mydatats_expfit.fittedvalues, label = "Forecasted Sales")
plt.legend()